# Phase III Code

Phase III will utilize Phase II Runoff Output to determine the dimensions of the percolation pond, as well as the anicut structure. The maximum runoff will be obtained from the data from Phase II. Using this Max runoff value, the volume of the percolation pond will be determined. The sixzing of the percolation pond will then infrom the dimensioning of the anicut structure. 

In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import os
import math

In [3]:
from units import unit
meter = unit('m')

## *Inputting Phase II Data*
***

First we want to import the runoff data from Phase II. We have imported the csv file. Depending on the format of the csv, the codes below clean up that data so that the annual runoff is provided.

In [4]:
Runoff = pd.read_csv('Precipitation.csv')
Runoff.head().T

#Reading the runoff data produced in Phase II

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


In [5]:
#Adding the summation of expected runoff for each upcoming year and adding it as its own column
annual = [ row.JAN + row.FEB + row.MAR + row.APR + row.MAY + row.JUN + row.JUL + row.AUG + row.SEP + row.OCT + row.NOV + row.DEC for index, row in Runoff.iterrows() ]
Runoff['ANNUAL'] = annual
Runoff.head().T

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


The code below allows us to obtain the [predicted] maximum runoff that will occur within the next 50 years. This maximum value allows us to know how much water our percolation pond must be able to hold.

In [6]:
#Choosing the maximum capacity yield projected in the next 50 years
Runoff['ANNUAL'].max()

768.8

# *Design of Percolation Pond*
***

Now, knowing the dimensions of the anicut, we can design for the dimensions of the pond behind it. We begin by reading in a file with some geographic information about the valley where the pond will be located. We need information about the slope of the land directly behind the dam.

In [7]:
PondSlope = pd.read_excel('Slope Elevation.xlsx')
PondSlope.head().T

,0,1,2,3,4
Distance from Dam (Southern/Soutwestern) (ft),0.0,1.0000,2.0000,3.0000,4.0000
Elevation (ft),1227.0,1227.0256,1227.0512,1227.0768,1227.1024


The slope is assumed to be constant as concluded by Google Earth Pro. The code below delivers the assumed slope per ft behind the dam. That slope, labeled "Elevation (ft)" must be input into the following theta equation to find the angle of the slope. 

In [8]:
slopes = PondSlope.apply(lambda x: np.polyfit(PondSlope.index, x, 1)[0])

# instructor code
xx = slopes[1]
print(xx)
#Assume that the slope is constant (because it is very close)
#Slope = 0.05 ft/ft



0.028791529411765476


In [9]:
theta = math.atan(xx)
theta

0.028783577766089783

The shape of the dam is assumed to be a volumetric triangular, due to the relatively low height. The assumption is that the slope on both sides of the pond ~0. The angle of the sloped ground behind the dam is used with simply geometry to compute the length maximum length behind that dam that can withold water from pouring over the dam. The second equation provides the length needed to withold the assumed annual runoff.

In [10]:
Length_of_Reservoir = Height/(math.tan(theta))
print('The length of the percolation pond is',round(Length_of_Reservoir,0), 'm')

Needed_Length = (2)*(Runoff['ANNUAL'].max())/(Height*Length)
print('The length needed for the percolation pond is', Needed_Length, 'm')

NameError: name 'Height' is not defined

From the code above, an if/else statement is used to decide if the suggested dimensions of the pond are acceptable given the predicted runoff. If dimenions are okay, model is complete. If dimenions will not hole the maximum water catchment, run model again with a different height.

In [ ]:
if Needed_Length <= Length_of_Reservoir:
    print('Dimensions of Reservoir are OK!')
    print('Height of Reservoir = ', Height, 'm')
    print('Width of Dam = ', Length, 'm')
    print('Length of Reservoir = ', round(Length_of_Reservoir,0), 'm')
else:
    print('Current Dimensions will not hold maximum water catchment.')
    print('Height of Reservoir = ', Height, 'm')
    print('Width of Dam = ', Length, 'm')
    print('Length of Reservoir = ', round(Length_of_Reservoir,0), 'm')

In [16]:
depth = 3.5

## *Design of Anicut Structure*
***

Now we want to begin to dimension the Anicut Structure. *An anicut is a dam built specifically to regulate water for irrigation*. We begin by reading in a file with some geographic information about the valley where the anicut will be built. We need information about the elevation of the valley, as well as the distance between two points of equal elevation on either side of the valley. 

In [26]:
AE = pd.read_csv('Anicut Elevation.csv')
AE

#AE = Anicut Elevation

,Elevation,Distance L,Distance R,Distance (English),Distance (Metric),Anicut Height (English),Anicut Height (Metric),Anicut Height (Rounded)
0,Ft,Ft,Ft,Ft,m,Ft,m,m
1,1228,606,742,136,41,1,0.30,1
2,1229,550,782,232,71,2,0.61,1
3,1230,495,823,328,100,3,0.91,1
4,1231,443,864,421,128,4,1.22,2
5,1232,393,903,510,155,5,1.52,2
6,1233,372,941,569,173,6,1.83,2
7,1234,353,969,616,188,7,2.13,3
8,1235,330,991,661,201,8,2.44,3
9,1236,310,1005,695,212,9,2.74,3


In [27]:
AE1 = AE.drop(['Distance L', 'Distance R', 'Distance (English)','Anicut Height (English)','Anicut Height (Rounded)'], axis=1)
AE1

,Elevation,Distance (Metric),Anicut Height (Metric)
0,Ft,m,m
1,1228,41,0.30
2,1229,71,0.61
3,1230,100,0.91
4,1231,128,1.22
5,1232,155,1.52
6,1233,173,1.83
7,1234,188,2.13
8,1235,201,2.44
9,1236,212,2.74


The table above records the change in the width of the valley at different elevation intervals. The geometric contraints of the valley will guide the length of the anicut, depending on the height of the *Freeboard* chosen. *The Freeboard is the height of a dam that is above the designed water level of the pond upstream of the dam*. **Below, choose a height for the freeboard of the anicut**. The freeboard needs to be in increments of 0.5m for the anicut to be economically built. The freeboad should be at a height to prevent overspill of any runoff event. The **code will determine the required length** of the anicut. 

In [28]:
print("Pick an anicut height between 0.5 m and 2.5m")
Freeboard_Height = float(input())
print('The Freeboard height is', meter(Freeboard_Height))

Pick an anicut height between 0.5 m and 2.5m
1
The Freeboard height is 1.00 m


In [29]:
if  Freeboard_Height == 0.5:
    Length = 71
    print('Length of Dam = 71 m')
elif Freeboard_Height == 1:
    Length = 128
    print('Length of Dam = 128 m')
elif Freeboard_Height == 1.5:
    Length = 155
    print('Length of Dam = 155 m')
elif Freeboard_Height == 2:
    Length = 188
    print('Length of Dam = 188 m')
elif Freeboard_Height == 2.5: 
    Length = 212
    print('Length of Dam = 212 m')

Length of Dam = 128 m


The freeboard height will be added to the **depth** of the percolation pond to obtain the **total height of the anicut**. 

In [30]:
Total_Height = Freeboard_Height + depth

Now that we have the *height* and *length* of the anicut, we can dimension the rest of the anicut. We want to find the **inclined height** and the **volume**. The proposed anicut strcture will be made of reinforced concrete. Unlike an earthen dam, concrete dams will not be subject to erosion on their super sturtcure. Therefore, erosion does not need to be accounted for in designing the face of the dam. 

The upstream face of the anicut will be vertical to intercept runoff. The downstream face is designed to have a slope to proivde support and stability to the face of the dam. 

To choose the slope of the downstream face of the anicut, please input the *rise(vertical)* and *run(horizontal)* of the dam's face

In [31]:
Horizontal = float(input())
Vertical = float(input())

print('The slope of the downstream face is', Horizontal, 'm Horizontal, by', Vertical, 'm Vertical')

0.7
1
The slope of the downstream face is 0.7 m Horizontal, by 1.0 m Vertical


In [32]:
Angle = math.atan(Vertical/Horizontal)*(180/math.pi)
print('The Angle for the inlcined face of the anicut is', Angle, 'Degrees')

The Angle for the inlcined face of the anicut is 55.00797980144134 Degrees


In [33]:
Downstream_Face_Length = Total_Height/math.sin(Angle*math.pi/180)
#Face_Length = The slope of the anicut will be the actual length along the face of the anicut
#Face_Length is in meters

print('The length of the face of the anicut is', meter(round(Downstream_Face_Length,2)))

The length of the face of the anicut is 5.49 m


In [34]:
Face_Surface_Area = Downstream_Face_Length*Length
print('The surface area of the face of the anicut is', round(Face_Surface_Area,2), 'meters squared')

The surface area of the face of the anicut is 703.1 meters squared


Now that we have the *height* and *length* of the anicut, we can determine the volume of the dam as well as the weight of the dam, using normal weight concrete. We assume the anicut's cross section is trapezoidal. In order to determine the volume we need to multiply the cross section of the anicut by the length. The equation for the cross section is that for a trapezoid. The volume of the anicut will multiplied by the unit weight of cocnrete to determine the weight of the anicut. 

In [35]:
Weight_Concrete = 2406.53 
#The weight of Normal Weight Concrete in (kg/m^3)

In [36]:
Crest = float(input())
#The crest is the length of the top part of the anicut structure
print('The crest of the anicut is', meter(Crest))

2
The crest of the anicut is 2.00 m


In [37]:
Cross_Section = Total_Height*(Crest+(Crest+(math.cos(Angle*math.pi/180)*Downstream_Face_Length)))/2
Volume_Anicut = Length*Cross_Section
Weight_Anicut = Volume_Anicut * Weight_Concrete 

In [38]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['Dimension', 'Value', 'Unit']),
                 cells=dict(values=[['Freeboard Height','Total Dam Height','Length of Dam','Crest Length','Cross Sectional Area','Volume of Dam','Weight of Dam'], 
                                    [float(Freeboard_Height), float(Total_Height), float(Length), float(Crest), float(round(Cross_Section,0)), float(round(Volume_Anicut,0)), float(round(Weight_Anicut,0))], 
                                    ['m','m','m','m','m^2','m^3','kg']]))
                     ])
fig.show()

End of Code.